<a href="https://colab.research.google.com/github/sohelmukadam/stir-fly/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 6.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=44db6a59753dc9349b7e0b20b300131eaec37706d1439696527fa9e404aa8766
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=045d93050bdcdfa805a45c15099d5187b6a372e59ee3e158d5fd9556cf643666
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [7]:
fashion_mnist=keras.datasets.fashion_mnist

In [10]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [12]:
train_images=train_images/255.0
test_images=test_images/255.0

In [13]:
train_images[0].shape

(28, 28)

In [16]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)

In [37]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation= 'relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
        kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
        activation= 'relu',
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units',min_value=32, max_value=128,step=16),
        activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')#output layer
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model



       


In [38]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters


In [39]:
tuner_search=RandomSearch(build_model,
                           objective= 'val_accuracy',
                           max_trials = 5,directory='output',project_name="Mnist Fashion")

In [40]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 11m 22s]
val_accuracy: 0.10316666960716248

Best val_accuracy So Far: 0.847000002861023
Total elapsed time: 01h 17m 29s
INFO:tensorflow:Oracle triggered exit


In [41]:
model=tuner_search.get_best_models(num_models=1)[0]

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        64576     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               4128880   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 4,195,706
Trainable params: 4,195,706
Non-trainable params: 0
_________________________________________________________________
